<a href="https://colab.research.google.com/github/zhulkhoirrifat/Mushroom-Classification/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python --version

Python 3.10.12


In [2]:
!pip install -U tensorflow==2.13.0 tfx==1.14.0 wheel==0.43.0 setuptools==69.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of apache-beam to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of apache-beam to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of apache-beam[gcp] to determine which version is

# Mushroom Clasiffication

## Import Library

In [3]:
import os
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

## Set Variable

In [4]:
PIPLINE_NAME = 'mushroom-pipline'
SCHEMA_PIPLINE_NAME = 'mushroom-tfdv-schema'

# Directory menyimpan artifact
PIPLINE_ROOT = os.path.join('piplines', PIPLINE_NAME)

# Path ke SQLite DB untuk digunakan sebagai MLMD storage
METADATA_PATH = os.path.join('metadata', PIPLINE_NAME, 'metadata.db')

# Direktori output dimana model yang dibuat dari pipline akan di ekspor
SERVING_MODEL_DIR = os.path.join('serving-model', PIPLINE_NAME)

In [5]:
DATA_ROOT = 'data'

interactive_context = InteractiveContext(pipeline_root=PIPLINE_ROOT)

## Data Ingestion

Membagi dataset untuk proses training ("train") dan evaluasi ("eval") dengan rasio 8:2.

In [6]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## Data Validation

### Summary Statistic (StatisticGen)

Membuat summary statistic menggunakan ```StatisticGen()```.

In [7]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples']
)
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [8]:
interactive_context.show(statistics_gen.outputs['statistics'])

### Data Schema

In [9]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics']
)
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [10]:
interactive_context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'cap-diameter',INT,required,,-
'cap-shape',INT,required,,-
'class',INT,required,,-
'gill-attachment',INT,required,,-
'gill-color',INT,required,,-
'season',FLOAT,required,,-
'stem-color',INT,required,,-
'stem-height',FLOAT,required,,-
'stem-width',INT,required,,-


### Example Validator

Mengidentifikasi anomali pada dataset menggunakan ```ExampleValidator()```.

In [11]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [12]:
interactive_context.show(example_validator.outputs['anomalies'])

## Data Preprocessing

### Transform

In [13]:
TRANSFORM_MODULE_FILE = "modules/mushroom-transform.py"

In [14]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft

CATEGORICAL_FEATURES = {
  "cap-shape": 7,
  "gill-attachment": 7,
  "gill-color": 12,
  "stem-color": 13,
}

NUMERICAL_FEATURES = [
  "cap-diameter",
  "stem-height",
  "stem-width"
]

LABEL_KEY = "class"

def transformed_name(key):
  """ Transform feature key

  Args:
    key(str): the key has to be transformed

  Returns:
    str: transformed key
  """

  return f"{key}_xf"

def preprocessing_fn(inputs):
  """Preprocess input features into transformed features

    Args:
      inputs (dict): map from feature keys to raw features

    Returns:
      dict: map from features keys to transformed features
  """
  outputs = {}

  for feature in NUMERICAL_FEATURES:
    outputs[transformed_name(feature)] = tft.scale_to_z_score(inputs[feature])

  for keys, values in CATEGORICAL_FEATURES.items():
    outputs[transformed_name(keys)] = tf.cast(inputs[keys], tf.int64)

  outputs[transformed_name(LABEL_KEY)] = inputs[LABEL_KEY]

  return outputs


Writing modules/mushroom-transform.py


In [15]:
transform = Transform(
    examples = example_gen.outputs['examples'],
    schema = schema_gen.outputs['schema'],
    module_file = os.path.abspath(TRANSFORM_MODULE_FILE)
)

interactive_context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### Tuner

In [16]:
TUNER_MODULE_FILE = "mushroom-tuner"

In [17]:
# %%writefile {TUNER_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft
import keras_tuner as kt
from keras_tuner.engine import base_tuner
from tfx.components.trainer.fn_args_utils import FnArgs
from typing import NamedTuple, Dict, Text, Any
from transform import (NUMERICAL_FEATURES, CATEGORICAL_FEATURES, LABEL_KEY, transformed_name)

NUM_EPOCHS = 10

TunerFnResult = NamedTuple("TunerFnResult", [
    ("tuner", base_tuner.BaseTuner),
    ("fit_kwargs", Dict[Text, Any])
])

def gzip_reader_fn(filenames):
  """ Loads compressed data

  Args:
    filenames(str): Path to the data directory

  Return:
    TfRecord: Compressed data

  """
  return tf.data.TFRecordDataset(filenames, compression_type="GZIP")

def input_fn(file_pattern, tf_transform_output, batch_size=64):
  transform_feature_spec = (
      tf_transform_output.transform_feature_spec().copy
  )

  dataset = tf.data.experimental.make_batched_features_dataset(
      file_pattern=file_pattern,
      batch_size=batch_size,
      features=transform_feature_spec,
      reader=gzip_reader_fn,
      label_key=LABEL_KEY
  )

  return dataset

def model_builder(hp):
  """ Build keras model with hyperparameter tuning

  Args:
    hp(kt.Hyperparameters): Hyperparameter object from keras tuner for determine hyperparameter

  Returns:
    tf.keras.Model: Keras Model that has been compiled.
  """
  num_hidden_layer = hp.Choice(
      "num_hidden_layer",
      values=[1, 2, 3, 4, 5]
  )

  dense_unit = hp.Int(
      "dense_unit",
      min_value=32,
      max_value=256,
      step=32
  )

  dropout_rate = hp.Float(
      "dropout_rate",
      min_value=0.1,
      max_value=0.5,
      step=0.1
  )

  learning_rate = hp.Choice(
      "learning_rate",
      values=[1e-2, 1e-3, 1e-4]
  )

  input_features = []

  for key, dims in CATEGORICAL_FEATURES.items():


  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(tf.keras.layers.Dense(units=hp_units, activation='relu'))


ModuleNotFoundError: No module named 'transform'